In [1]:
import os, time, re, json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from scipy.special import softmax
import joblib
from joblib import Parallel, delayed
from sklearn.tree import DecisionTreeClassifier
from lightgbm import Dataset
import lightgbm
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import  RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.base import BaseEstimator, TransformerMixin

### This loads all of the ENS ranked subsets and trains 499 models, one for each subset.

In [2]:
# Points to the training data.
train_dir = "../data/train-subsets"

# Creates subset CSV list.
csv_files = [f'X_train_id_top_{i}_features.csv' for i in range(1, 500)]

# Init dictionary to store dataframes.
dfs = {}

# Iteratively load CSVs.
for feature_number in tqdm(range(1, 500), desc="Loading CSV files"):
    df_name = f"df{feature_number}"
    file_path = os.path.join(train_dir, f'X_train_id_top_{feature_number}_features.csv')
    df = pd.read_csv(file_path, low_memory=False)
    dfs[df_name] = df

print(f"Loaded {len(dfs)} dataframes.")
print(dfs.keys())

Loading CSV files:   0%|          | 0/499 [00:00<?, ?it/s]

Loaded 499 dataframes.
dict_keys(['df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8', 'df9', 'df10', 'df11', 'df12', 'df13', 'df14', 'df15', 'df16', 'df17', 'df18', 'df19', 'df20', 'df21', 'df22', 'df23', 'df24', 'df25', 'df26', 'df27', 'df28', 'df29', 'df30', 'df31', 'df32', 'df33', 'df34', 'df35', 'df36', 'df37', 'df38', 'df39', 'df40', 'df41', 'df42', 'df43', 'df44', 'df45', 'df46', 'df47', 'df48', 'df49', 'df50', 'df51', 'df52', 'df53', 'df54', 'df55', 'df56', 'df57', 'df58', 'df59', 'df60', 'df61', 'df62', 'df63', 'df64', 'df65', 'df66', 'df67', 'df68', 'df69', 'df70', 'df71', 'df72', 'df73', 'df74', 'df75', 'df76', 'df77', 'df78', 'df79', 'df80', 'df81', 'df82', 'df83', 'df84', 'df85', 'df86', 'df87', 'df88', 'df89', 'df90', 'df91', 'df92', 'df93', 'df94', 'df95', 'df96', 'df97', 'df98', 'df99', 'df100', 'df101', 'df102', 'df103', 'df104', 'df105', 'df106', 'df107', 'df108', 'df109', 'df110', 'df111', 'df112', 'df113', 'df114', 'df115', 'df116', 'df117', 'df118', 'df119', 'df

In [3]:
# These are the original LoGoFunc LightGBM hyperparameters.
params = {'num_iterations_0': 800, 'num_leaves_0': 1042, 'learning_rate_0': 0.238671566534306, 'min_child_weight_0': 6.766558232036225, 'min_data_in_leaf_0': 93, 'max_depth_0': 5, 'colsample_bytree_0': 0.6688859597860277, 'subsample_0': 0.8312388549172839, 'reg_lambda_0': 66, 'reg_alpha_0': 16, 'is_unbalance_0': 0, 'num_iterations_1': 350, 'num_leaves_1': 1622, 'learning_rate_1': 0.028890291618521663, 'min_child_weight_1': 1.29470594717391, 'min_data_in_leaf_1': 7, 'max_depth_1': 8, 'colsample_bytree_1': 0.5340170892793712, 'subsample_1': 0.5950224961130018, 'reg_lambda_1': 49, 'reg_alpha_1': 28, 'is_unbalance_1': 0, 'num_iterations_2': 800, 'num_leaves_2': 2222, 'learning_rate_2': 0.17174599284942418, 'min_child_weight_2': 1.8005712053303213, 'min_data_in_leaf_2': 8, 'max_depth_2': 15, 'colsample_bytree_2': 0.5019614361672425, 'subsample_2': 0.9430371009725439, 'reg_lambda_2': 77, 'reg_alpha_2': 59, 'is_unbalance_2': 0, 'num_iterations_3': 850, 'num_leaves_3': 1402, 'learning_rate_3': 0.06921991285839768, 'min_child_weight_3': 13.087995286677666, 'min_data_in_leaf_3': 56, 'max_depth_3': 3, 'colsample_bytree_3': 0.8727149472571005, 'subsample_3': 0.5270851576566667, 'reg_lambda_3': 62, 'reg_alpha_3': 14, 'is_unbalance_3': 0, 'num_iterations_4': 650, 'num_leaves_4': 2782, 'learning_rate_4': 0.18709721607289698, 'min_child_weight_4': 17.54726411044243, 'min_data_in_leaf_4': 8, 'max_depth_4': 19, 'colsample_bytree_4': 0.6822272014840585, 'subsample_4': 0.7871261765303349, 'reg_lambda_4': 30, 'reg_alpha_4': 35, 'is_unbalance_4': 0, 'num_iterations_5': 900, 'num_leaves_5': 2742, 'learning_rate_5': 0.25460052667719146, 'min_child_weight_5': 12.687734323659901, 'min_data_in_leaf_5': 45, 'max_depth_5': 12, 'colsample_bytree_5': 0.4205270360424994, 'subsample_5': 0.631485175697527, 'reg_lambda_5': 33, 'reg_alpha_5': 26, 'is_unbalance_5': 0, 'num_iterations_6': 400, 'num_leaves_6': 1202, 'learning_rate_6': 0.11205962494027148, 'min_child_weight_6': 14.223014265948846, 'min_data_in_leaf_6': 7, 'max_depth_6': 7, 'colsample_bytree_6': 0.8179730880954434, 'subsample_6': 0.5229839192948834, 'reg_lambda_6': 94, 'reg_alpha_6': 87, 'is_unbalance_6': 1, 'num_iterations_7': 150, 'num_leaves_7': 662, 'learning_rate_7': 0.10924921778438157, 'min_child_weight_7': 19.492528675644262, 'min_data_in_leaf_7': 79, 'max_depth_7': 12, 'colsample_bytree_7': 0.6331150723036664, 'subsample_7': 0.5473894697168075, 'reg_lambda_7': 30, 'reg_alpha_7': 2, 'is_unbalance_7': 0, 'num_iterations_8': 750, 'num_leaves_8': 1662, 'learning_rate_8': 0.03379525849843262, 'min_child_weight_8': 10.368747400877217, 'min_data_in_leaf_8': 89, 'max_depth_8': 8, 'colsample_bytree_8': 0.45913859590099626, 'subsample_8': 0.7892755293794346, 'reg_lambda_8': 70, 'reg_alpha_8': 88, 'is_unbalance_8': 1, 'num_iterations_9': 950, 'num_leaves_9': 862, 'learning_rate_9': 0.08614934353267474, 'min_child_weight_9': 13.020085530836386, 'min_data_in_leaf_9': 31, 'max_depth_9': 9, 'colsample_bytree_9': 0.7826235420161084, 'subsample_9': 0.45084273310866746, 'reg_lambda_9': 71, 'reg_alpha_9': 15, 'is_unbalance_9': 0, 'num_iterations_10': 550, 'num_leaves_10': 1022, 'learning_rate_10': 0.27998414099470914, 'min_child_weight_10': 16.621816111130492, 'min_data_in_leaf_10': 48, 'max_depth_10': 10, 'colsample_bytree_10': 0.8803288365381602, 'subsample_10': 0.9139447258450247, 'reg_lambda_10': 67, 'reg_alpha_10': 24, 'is_unbalance_10': 0, 'num_iterations_11': 750, 'num_leaves_11': 1062, 'learning_rate_11': 0.26675566671115014, 'min_child_weight_11': 3.2422294926713744, 'min_data_in_leaf_11': 53, 'max_depth_11': 3, 'colsample_bytree_11': 0.5240179776958962, 'subsample_11': 0.7596911313371446, 'reg_lambda_11': 41, 'reg_alpha_11': 92, 'is_unbalance_11': 0, 'num_iterations_12': 950, 'num_leaves_12': 2402, 'learning_rate_12': 0.19391455848714093, 'min_child_weight_12': 7.247198130957093, 'min_data_in_leaf_12': 54, 'max_depth_12': 5, 'colsample_bytree_12': 0.7168420201679513, 'subsample_12': 0.5892195277276672, 'reg_lambda_12': 35, 'reg_alpha_12': 9, 'is_unbalance_12': 0, 'num_iterations_13': 1000, 'num_leaves_13': 1322, 'learning_rate_13': 0.08708808854407579, 'min_child_weight_13': 9.660370330049789, 'min_data_in_leaf_13': 34, 'max_depth_13': 8, 'colsample_bytree_13': 0.6875964449420523, 'subsample_13': 0.851851576012825, 'reg_lambda_13': 44, 'reg_alpha_13': 45, 'is_unbalance_13': 0, 'num_iterations_14': 850, 'num_leaves_14': 2382, 'learning_rate_14': 0.15787599914959588, 'min_child_weight_14': 19.422819035703707, 'min_data_in_leaf_14': 72, 'max_depth_14': 16, 'colsample_bytree_14': 0.6625046547799455, 'subsample_14': 0.7724363626558111, 'reg_lambda_14': 5, 'reg_alpha_14': 39, 'is_unbalance_14': 0, 'num_iterations_15': 650, 'num_leaves_15': 2382, 'learning_rate_15': 0.14923043478058995, 'min_child_weight_15': 7.981196137920284, 'min_data_in_leaf_15': 18, 'max_depth_15': 15, 'colsample_bytree_15': 0.6643095735704215, 'subsample_15': 0.7929296085894856, 'reg_lambda_15': 10, 'reg_alpha_15': 55, 'is_unbalance_15': 0, 'num_iterations_16': 1000, 'num_leaves_16': 1362, 'learning_rate_16': 0.1134325274584146, 'min_child_weight_16': 13.721587805199734, 'min_data_in_leaf_16': 68, 'max_depth_16': 17, 'colsample_bytree_16': 0.8226636316974583, 'subsample_16': 0.8302625124116251, 'reg_lambda_16': 77, 'reg_alpha_16': 51, 'is_unbalance_16': 1, 'num_iterations_17': 700, 'num_leaves_17': 1322, 'learning_rate_17': 0.07893469179626852, 'min_child_weight_17': 11.924417575644181, 'min_data_in_leaf_17': 14, 'max_depth_17': 0, 'colsample_bytree_17': 0.829372086801561, 'subsample_17': 0.5869237894432597, 'reg_lambda_17': 76, 'reg_alpha_17': 11, 'is_unbalance_17': 0, 'num_iterations_18': 600, 'num_leaves_18': 2522, 'learning_rate_18': 0.08861952240067116, 'min_child_weight_18': 5.0671335947879355, 'min_data_in_leaf_18': 36, 'max_depth_18': 11, 'colsample_bytree_18': 0.645303203711488, 'subsample_18': 0.5623140555048293, 'reg_lambda_18': 91, 'reg_alpha_18': 30, 'is_unbalance_18': 1, 'num_iterations_19': 350, 'num_leaves_19': 802, 'learning_rate_19': 0.12549472119181668, 'min_child_weight_19': 11.126359710312927, 'min_data_in_leaf_19': 78, 'max_depth_19': 8, 'colsample_bytree_19': 0.71048285192183, 'subsample_19': 0.8618184095019077, 'reg_lambda_19': 5, 'reg_alpha_19': 47, 'is_unbalance_19': 1, 'num_iterations_20': 300, 'num_leaves_20': 1882, 'learning_rate_20': 0.29333163899611775, 'min_child_weight_20': 9.284775212037633, 'min_data_in_leaf_20': 13, 'max_depth_20': 1, 'colsample_bytree_20': 0.8834089058116563, 'subsample_20': 0.7546606659132303, 'reg_lambda_20': 78, 'reg_alpha_20': 29, 'is_unbalance_20': 1, 'num_iterations_21': 850, 'num_leaves_21': 2902, 'learning_rate_21': 0.08799353684452318, 'min_child_weight_21': 17.82306953970517, 'min_data_in_leaf_21': 82, 'max_depth_21': 14, 'colsample_bytree_21': 0.6620197361288442, 'subsample_21': 0.803455126555435, 'reg_lambda_21': 61, 'reg_alpha_21': 61, 'is_unbalance_21': 0, 'num_iterations_22': 550, 'num_leaves_22': 1882, 'learning_rate_22': 0.028528704175685697, 'min_child_weight_22': 0.11852372928468169, 'min_data_in_leaf_22': 47, 'max_depth_22': 15, 'colsample_bytree_22': 0.9829653861819089, 'subsample_22': 0.6302173102891655, 'reg_lambda_22': 45, 'reg_alpha_22': 61, 'is_unbalance_22': 0, 'num_iterations_23': 600, 'num_leaves_23': 1942, 'learning_rate_23': 0.06942766069966466, 'min_child_weight_23': 16.4898364622421, 'min_data_in_leaf_23': 5, 'max_depth_23': 17, 'colsample_bytree_23': 0.9175122476331665, 'subsample_23': 0.7157705887348372, 'reg_lambda_23': 98, 'reg_alpha_23': 57, 'is_unbalance_23': 0, 'num_iterations_24': 400, 'num_leaves_24': 362, 'learning_rate_24': 0.08761207485753396, 'min_child_weight_24': 19.444295883810717, 'min_data_in_leaf_24': 82, 'max_depth_24': -1, 'colsample_bytree_24': 0.9282327198957283, 'subsample_24': 0.4750980815621907, 'reg_lambda_24': 66, 'reg_alpha_24': 50, 'is_unbalance_24': 0, 'num_iterations_25': 100, 'num_leaves_25': 1382, 'learning_rate_25': 0.04878946427510325, 'min_child_weight_25': 0.11755350203460324, 'min_data_in_leaf_25': 34, 'max_depth_25': 17, 'colsample_bytree_25': 0.6535167320212393, 'subsample_25': 0.5850886386281302, 'reg_lambda_25': 95, 'reg_alpha_25': 30, 'is_unbalance_25': 0, 'num_iterations_26': 900, 'num_leaves_26': 1702, 'learning_rate_26': 0.04892682557831036, 'min_child_weight_26': 2.9843196522546362, 'min_data_in_leaf_26': 96, 'max_depth_26': 19, 'colsample_bytree_26': 0.6160464301457169, 'subsample_26': 0.8805506079285266, 'reg_lambda_26': 28, 'reg_alpha_26': 53, 'is_unbalance_26': 0}

In [ ]:
""" Iterate through the entire training script for each subeset DataFrame in the dictionary. """

train_dfs = dfs
for df_name, X_train in train_dfs.items():
    y_train = pd.read_csv('../data/y_train_id.csv')
    print(y_train.shape)

    # Encoding the labels.
    y_train_enc = []
    for lab in y_train['label']:
        if lab == 'GOF':
            y_train_enc.append(1)
        elif lab == 'LOF':
            y_train_enc.append(2)
        else:
            y_train_enc.append(0)
    y_train = y_train_enc
    print(len(y_train))

    print(f"Processing {df_name}")

    model_name = f"lgbm_{df_name}"
    model_directory = f'../LightGBM/models/{model_name}'

    # Create the directory.
    if not os.path.exists(model_directory):
        os.makedirs(model_directory)

    print(f"Directory '{model_directory}' is ready.")

    # These lists are used to determine imputation strategies for the specified features (inherited from LoGoFunc)
    NEGONE_FEATURES = ['MOD_RES','REGION','INTERACTION_REGION','REQUIRED_FOR_INTER','ATP_binding_gbind','Ca2+_binding_gbind','DNA_binding_gbind','HEME_binding_gbind','Mg2+_binding_gbind','Mn2+_binding_gbind','RNA_binding_gbind','Dist2Mutation','BLOSUM62','ProteinLengthChange','TSSDistance','1000Gp3_AF','UK10K_AF','gnomAD_exomes_AF','gnomAD_genomes_AF','MSC_95CI','rel_cDNA_pos','rel_CDS_pos','rel_prot_pos','GDI','Selective_pressure','Clarks_distance','CDS_len','Number_of_paralogs','denovo_Zscore','RVIS','Indispensability_score','RSA','ASA','RSA_Zfit','before_RSA_3','before_RSA_8','before_RSA_15','after_RSA_3','after_RSA_8','after_RSA_15','before_ASA_3','before_ASA_8','before_ASA_15','after_ASA_3','after_ASA_8','after_ASA_15','Phosphorylation','Acetylation','Methylation','Ubiquitination','Glycosylation','PTM','AF_Relative_ASA','IUPRED2','ANCHOR2','before_IUPRED_3','before_IUPRED_8','before_IUPRED_15','after_IUPRED_3','after_IUPRED_8','after_IUPRED_15','before_ANCHOR_3','before_ANCHOR_8','before_ANCHOR_15','after_ANCHOR_3','after_ANCHOR_8','after_ANCHOR_15','A3D_SCORE','n_contacts','distance_com','concavity_score','S_DDG[SEQ]','S_DDG[3D]','hgmd_mutcount','gnomsingle_mutcount','gnom_mutcount','AF_confidence','isHomomultimer','num_interactions','ppi_combined_0','ppi_combined_1','ppi_combined_2','ppi_combined_3','ppi_combined_4','ppi_combined_5','ppi_combined_6','ppi_combined_7','ppi_combined_8','ppi_combined_9','ppi_combined_10','ppi_combined_11','ppi_combined_12','ppi_combined_13','ppi_combined_14','ppi_combined_15','ppi_combined_16','ppi_combined_17','ppi_combined_18','ppi_combined_19','ppi_combined_20','ppi_combined_21','ppi_combined_22','ppi_combined_23','ppi_combined_24','ppi_combined_25','ppi_combined_26','ppi_combined_27','ppi_combined_28','ppi_combined_29','ppi_combined_30','ppi_combined_31','ppi_combined_32','ppi_combined_33','ppi_combined_34','ppi_combined_35','ppi_combined_36','ppi_combined_37','ppi_combined_38','ppi_combined_39','ppi_combined_40','ppi_combined_41','ppi_combined_42','ppi_combined_43','ppi_combined_44','ppi_combined_45','ppi_combined_46','ppi_combined_47','ppi_combined_48','ppi_combined_49','ppi_combined_50','ppi_combined_51','ppi_combined_52','ppi_combined_53','ppi_combined_54','ppi_combined_55','ppi_combined_56','ppi_combined_57','ppi_combined_58','ppi_combined_59','ppi_combined_60','ppi_combined_61','ppi_combined_62','ppi_combined_63','s_het','DRNApredDNAscore_aa_window_3_prev','DRNApredDNAscore_aa_window_8_prev','DRNApredDNAscore_aa_window_15_prev','DRNApredDNAscore_aa_window_3_next','DRNApredDNAscore_aa_window_8_next','DRNApredDNAscore_aa_window_15_next','DRNApredDNAscore_aa','ASAquick_normscore_aa_window_3_prev','ASAquick_normscore_aa_window_8_prev','ASAquick_normscore_aa_window_15_prev','ASAquick_normscore_aa_window_3_next','ASAquick_normscore_aa_window_8_next','ASAquick_normscore_aa_window_15_next','ASAquick_normscore_aa','ASAquick_rawscore_aa_window_3_prev','ASAquick_rawscore_aa_window_8_prev','ASAquick_rawscore_aa_window_15_prev','ASAquick_rawscore_aa_window_3_next','ASAquick_rawscore_aa_window_8_next','ASAquick_rawscore_aa_window_15_next','ASAquick_rawscore_aa','DFLpredScore_aa_window_3_prev','DFLpredScore_aa_window_8_prev','DFLpredScore_aa_window_15_prev','DFLpredScore_aa_window_3_next','DFLpredScore_aa_window_8_next','DFLpredScore_aa_window_15_next','DFLpredScore_aa','DRNApredRNAscore_aa_window_3_prev','DRNApredRNAscore_aa_window_8_prev','DRNApredRNAscore_aa_window_15_prev','DRNApredRNAscore_aa_window_3_next','DRNApredRNAscore_aa_window_8_next','DRNApredRNAscore_aa_window_15_next','DRNApredRNAscore_aa','DisoDNAscore_aa_window_3_prev','DisoDNAscore_aa_window_8_prev','DisoDNAscore_aa_window_15_prev','DisoDNAscore_aa_window_3_next','DisoDNAscore_aa_window_8_next','DisoDNAscore_aa_window_15_next','DisoDNAscore_aa','DisoPROscore_aa_window_3_prev','DisoPROscore_aa_window_8_prev','DisoPROscore_aa_window_15_prev','DisoPROscore_aa_window_3_next','DisoPROscore_aa_window_8_next','DisoPROscore_aa_window_15_next','DisoPROscore_aa','DisoRNAscore_aa_window_3_prev','DisoRNAscore_aa_window_8_prev','DisoRNAscore_aa_window_15_prev','DisoRNAscore_aa_window_3_next','DisoRNAscore_aa_window_8_next','DisoRNAscore_aa_window_15_next','DisoRNAscore_aa','MMseq2_conservation_level_aa_window_3_prev','MMseq2_conservation_level_aa_window_8_prev','MMseq2_conservation_level_aa_window_15_prev','MMseq2_conservation_level_aa_window_3_next','MMseq2_conservation_level_aa_window_8_next','MMseq2_conservation_level_aa_window_15_next','MMseq2_conservation_level_aa','MMseq2_conservation_score_aa_window_3_prev','MMseq2_conservation_score_aa_window_8_prev','MMseq2_conservation_score_aa_window_15_prev','MMseq2_conservation_score_aa_window_3_next','MMseq2_conservation_score_aa_window_8_next','MMseq2_conservation_score_aa_window_15_next','MMseq2_conservation_score_aa','MoRFchibiScore_aa_window_3_prev','MoRFchibiScore_aa_window_8_prev','MoRFchibiScore_aa_window_15_prev','MoRFchibiScore_aa_window_3_next','MoRFchibiScore_aa_window_8_next','MoRFchibiScore_aa_window_15_next','MoRFchibiScore_aa','PSIPRED_helix_aa_window_3_prev','PSIPRED_helix_aa_window_8_prev','PSIPRED_helix_aa_window_15_prev','PSIPRED_helix_aa_window_3_next','PSIPRED_helix_aa_window_8_next','PSIPRED_helix_aa_window_15_next','PSIPRED_helix_aa','PSIPRED_strand_aa_window_3_prev','PSIPRED_strand_aa_window_8_prev','PSIPRED_strand_aa_window_15_prev','PSIPRED_strand_aa_window_3_next','PSIPRED_strand_aa_window_8_next','PSIPRED_strand_aa_window_15_next','PSIPRED_strand_aa','SCRIBERscore_aa_window_3_prev','SCRIBERscore_aa_window_8_prev','SCRIBERscore_aa_window_15_prev','SCRIBERscore_aa_window_3_next','SCRIBERscore_aa_window_8_next','SCRIBERscore_aa_window_15_next','SCRIBERscore_aa','SignalP_score_aa_window_3_prev','SignalP_score_aa_window_8_prev','SignalP_score_aa_window_15_prev','SignalP_score_aa_window_3_next','SignalP_score_aa_window_8_next','SignalP_score_aa_window_15_next','SignalP_score_aa','gtex_Adipose_-_Subcutaneous','gtex_Adipose_-_Visceral_(Omentum)','gtex_Adrenal_Gland','gtex_Artery_-_Aorta','gtex_Artery_-_Coronary','gtex_Artery_-_Tibial','gtex_Bladder','gtex_Brain_-_Amygdala','gtex_Brain_-_Anterior_cingulate_cortex_(BA24)','gtex_Brain_-_Caudate_(basal_ganglia)','gtex_Brain_-_Cerebellar_Hemisphere','gtex_Brain_-_Cerebellum','gtex_Brain_-_Cortex','gtex_Brain_-_Frontal_Cortex_(BA9)','gtex_Brain_-_Hippocampus','gtex_Brain_-_Hypothalamus','gtex_Brain_-_Nucleus_accumbens_(basal_ganglia)','gtex_Brain_-_Putamen_(basal_ganglia)','gtex_Brain_-_Spinal_cord_(cervical_c-1)','gtex_Brain_-_Substantia_nigra','gtex_Breast_-_Mammary_Tissue','gtex_Cells_-_Cultured_fibroblasts','gtex_Cells_-_EBV-transformed_lymphocytes','gtex_Cervix_-_Ectocervix','gtex_Cervix_-_Endocervix','gtex_Colon_-_Sigmoid','gtex_Colon_-_Transverse','gtex_Esophagus_-_Gastroesophageal_Junction','gtex_Esophagus_-_Mucosa','gtex_Esophagus_-_Muscularis','gtex_Fallopian_Tube','gtex_Heart_-_Atrial_Appendage','gtex_Heart_-_Left_Ventricle','gtex_Kidney_-_Cortex','gtex_Kidney_-_Medulla','gtex_Liver','gtex_Lung','gtex_Minor_Salivary_Gland','gtex_Muscle_-_Skeletal','gtex_Nerve_-_Tibial','gtex_Ovary','gtex_Pancreas','gtex_Pituitary','gtex_Prostate','gtex_Skin_-_Not_Sun_Exposed_(Suprapubic)','gtex_Skin_-_Sun_Exposed_(Lower_leg)','gtex_Small_Intestine_-_Terminal_Ileum','gtex_Spleen','gtex_Stomach','gtex_Testis','gtex_Thyroid','gtex_Uterus','gtex_Vagina','gtex_Whole_Blood','haplo','haplo_imputed','PHOSPHORYLATION','ACETYLATION','UBIQUITINATION','S-NITROSYLATION','N-GLYCOSYLATION','METHYLATION','O-GLYCOSYLATION','MYRISTOYLATION','C-GLYCOSYLATION','SUMOYLATION','S-GLYCOSYLATION','polyphen_nobs','polyphen_normasa','polyphen_dvol','polyphen_dprop','polyphen_bfact','polyphen_hbonds','polyphen_avenhet','polyphen_mindhet','polyphen_avenint','polyphen_mindint','polyphen_avensit','polyphen_mindsit','polyphen_idpmax','polyphen_idpsnp','polyphen_idqmin','motifECount','motifEHIPos','motifEScoreChng','Dst2Splice','motifDist','EncodeH3K4me1-sum','EncodeH3K4me1-max','EncodeH3K4me2-sum','EncodeH3K4me2-max','EncodeH3K4me3-sum','EncodeH3K4me3-max','EncodeH3K9ac-sum','EncodeH3K9ac-max','EncodeH3K9me3-sum','EncodeH3K9me3-max','EncodeH3K27ac-sum','EncodeH3K27ac-max','EncodeH3K27me3-sum','EncodeH3K27me3-max','EncodeH3K36me3-sum','EncodeH3K36me3-max','EncodeH3K79me2-sum','EncodeH3K79me2-max','EncodeH4K20me1-sum','EncodeH4K20me1-max','EncodeH2AFZ-sum','EncodeH2AFZ-max','EncodeDNase-sum','EncodeDNase-max','EncodetotalRNA-sum','EncodetotalRNA-max','Grantham_x','Freq100bp','Rare100bp','Sngl100bp','Freq1000bp','Rare1000bp','Sngl1000bp','Freq10000bp','Rare10000bp','Sngl10000bp','RemapOverlapTF','RemapOverlapCL','Charge','Volume','Hydrophobicity','Polarity','Ex','PAM250','JM','HGMD2003','VB','Transition','COSMIC','COSMICvsSWISSPROT','HAPMAP','COSMICvsHAPMAP',]
    MEDIAN_FEATURES = ['CADD_raw','Conservation','MaxEntScan_alt','MaxEntScan_diff','MaxEntScan_ref','ada_score','rf_score','FATHMM_score','GERPplus_plus_NR','GERPplus_plus_RS','GM12878_fitCons_score','GenoCanyon_score','H1_hESC_fitCons_score','HUVEC_fitCons_score','LINSIGHT','LIST_S2_score','LRT_score','M_CAP_score','MPC_score','MVP_score','MutationAssessor_score','MutationTaster_score','PROVEAN_score','SiPhy_29way_logOdds','VEST4_score','fathmm_MKL_coding_score','fathmm_XF_coding_score','integrated_fitCons_score','phastCons100way_vertebrate','phastCons17way_primate','phastCons30way_mammalian','phyloP100way_vertebrate','phyloP17way_primate','phyloP30way_mammalian','Condel_score','SIFT_score','NearestExonJB_distance','NearestExonJB_len','Dominant_probability','Recessive_probability','polyphen_dscore','polyphen_score1','polyphen_score2','ConsScore','GC','CpG','minDistTSS','minDistTSE','priPhCons','mamPhCons','verPhCons','priPhyloP','mamPhyloP','verPhyloP','bStatistic_y','targetScan','mirSVR-Score','mirSVR-E','mirSVR-Aln','cHmm_E1','cHmm_E2','cHmm_E3','cHmm_E4','cHmm_E5','cHmm_E6','cHmm_E7','cHmm_E8','cHmm_E9','cHmm_E10','cHmm_E11','cHmm_E12','cHmm_E13','cHmm_E14','cHmm_E15','cHmm_E16','cHmm_E17','cHmm_E18','cHmm_E19','cHmm_E20','cHmm_E21','cHmm_E22','cHmm_E23','cHmm_E24','cHmm_E25','GerpRS','GerpRSpval','GerpN','GerpS','tOverlapMotifs','SpliceAI-acc-gain','SpliceAI-acc-loss','SpliceAI-don-gain','SpliceAI-don-loss','MMSp_acceptorIntron','MMSp_acceptor','MMSp_exon','MMSp_donor','MMSp_donorIntron','dbscSNV-ada_score','dbscSNV-rf_score',]

    # Filter NEGONE_FEATURES and MEDIAN_FEATURES based on current X_train columns.    
    negone_features_filtered = [feature for feature in NEGONE_FEATURES if feature in X_train.columns]
    median_features_filtered = [feature for feature in MEDIAN_FEATURES if feature in X_train.columns]

    # Redefine NEGONE_FEATURES and MEDIAN_FEATURES with updated values.
    NEGONE_FEATURES = negone_features_filtered
    MEDIAN_FEATURES = median_features_filtered

    print("Filtered NEGONE_FEATURES:", NEGONE_FEATURES)
    print("Filtered MEDIAN_FEATURES:", MEDIAN_FEATURES)

    def generate_preprocessor(numeric_features, categorical_features, N_JOBS, cat_encode_type, 
                                do_specificimpute, do_featureselection, 
                                do_sampling, do_pca, var_thresh, oversample_technique, 
                                negone_features=NEGONE_FEATURES, median_features=MEDIAN_FEATURES,
                                prefix='', do_feature_subset=False, max_features=1, do_removeppi=False, do_removegtex=False):
        cat_encoders = [OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=-1), 
                        OneHotEncoder(sparse=False, handle_unknown='infrequent_if_exist', min_frequency=10)]
        categorical_transformer = cat_encoders[cat_encode_type]

        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', MinMaxScaler(feature_range =(0, 1), clip=True))])

        median_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', MinMaxScaler(feature_range =(0, 1), clip=True))])

        negone_transformer = Pipeline(steps=[
            ('scaler', MinMaxScaler(feature_range =(0, 1), clip=True)),
            ('imputer', SimpleImputer(strategy='constant', fill_value=-1)),
        ])

        preprocessor = None
        if do_specificimpute:
            preprocessor = ColumnTransformer(
                transformers=[
                    ('median', median_transformer, median_features),
                    ('negone', negone_transformer, negone_features),
                    ('cat', categorical_transformer, categorical_features),
            ])
        else:
            preprocessor = ColumnTransformer(
                transformers=[
                    ('numeric', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features),
                ])

        vt = VarianceThreshold(threshold=var_thresh)
        steps = [('initial', preprocessor), ('removeba', RemoveBeforeAfterTransformer()), ('variance_threshold', vt)]
        if do_sampling == 1:
            steps.append(('undersampling', RandomUnderSampler(random_state=42)))
        if do_sampling == 2:
            oversamplers = [SMOTE(n_jobs=N_JOBS,random_state=42), RandomOverSampler(random_state=42)]
            steps.append(('oversampling', oversamplers[oversample_technique]))
        if do_pca:
            steps.append(('pca', PCA()))

        preprocessor = Pipeline(steps=steps)
        return preprocessor

    class RemoveBeforeAfterTransformer(BaseEstimator, TransformerMixin):
        def __init__(self):
            self.drop_cols = None

        def fit(self, X, y=None):
            print(X.shape)
            self.drop_cols = X.columns[X.columns.str.contains('after|before|gnomAD_exomes_AF|gnomAD_genomes_AF', regex=True)]
            return self

        def transform(self, X, y=None):
            X = X.drop(columns=self.drop_cols)
            print(X.shape)
            return X

        def get_feature_names_out(self, input_features=None):
            return [f for f in input_features if f not in self.drop_cols]


    def preprocess(preprocessor, train_data, train_labels, quiet=False):
        for k, v in preprocessor.steps:
            print(f"Applying step: {k}")
            print(f"Input shape: {train_data.shape}")
            print(f"Input columns: {train_data.columns}")
            
            if k == 'initial':
                start = time.time()
                v.fit(train_data)
                train_data = pd.DataFrame(v.transform(train_data), columns=v.get_feature_names_out())
                end = time.time()
                if not quiet:
                    print(k + ' took ' + str(end - start) + ' to run.')
            elif k == 'oversampling' or k == 'undersampling':
                start = time.time()
                input_features = train_data.columns
                train_data, train_labels = v.fit_resample(train_data, train_labels)
                train_data = pd.DataFrame(train_data, columns=input_features)
                end = time.time()
                if not quiet:
                    print(k + ' took ' + str(end - start) + ' to run.')
            else:
                start = time.time()
                v.fit(train_data)
                input_features = train_data.columns
                train_data = pd.DataFrame(v.transform(train_data), columns=v.get_feature_names_out(input_features))
                end = time.time()
                if not quiet:
                    print(k + ' took ' + str(end - start) + ' to run.')
            
            print(f"Output shape: {train_data.shape}")
            print(f"Output columns: {train_data.columns}")
            print("---")

        for col in train_data.columns:
            try:
                train_data[col] = train_data[col].astype('float')
            except:
                train_data[col] = train_data[col].astype('category')

        return train_data, train_labels

    def transform(test_data, preprocessor, quiet=False):
        for k, v in preprocessor.steps:
            if k == 'initial':
                test_data = pd.DataFrame(v.transform(test_data), columns=v.get_feature_names_out())
            elif k == 'oversampling' or k == 'undersampling':
                continue
            else:
                test_data = v.transform(test_data)
        test_data = pd.DataFrame(test_data)

        for col in test_data.columns:
            try:
                test_data[col] = test_data[col].astype('float')
            except:
                test_data[col] = test_data[col].astype('category')

        return test_data.to_numpy()

    # Specify the paths for model, results, and metrics.
    model_path = model_directory
    output_path = f'../LightGBM/results/{model_name}-validation.csv'
    metrics_file = f'../LightGBM/metrics/{model_name}-validation-metrics.csv'

    # This is used to drop columns that only contain NaN values.
    def drop_allnan(data):
        for col in data.columns:
            if data[col].isna().sum() == len(data):
                data = data.drop(columns=col)
        return data

    # This encodes the IMPACT feature's column.
    X_train = drop_allnan(X_train)
    impact_vals = {'LOW': 0, 'MODIFIER': 1, 'MODERATE': 1.5, 'HIGH': 2}

    for col in X_train.columns:
        if X_train[col].dtype == 'object':
            if set(X_train[col].unique()) <= set(impact_vals.keys()):
                X_train[col] = X_train[col].map(impact_vals).fillna(0)

    # Conditional imputation based on feature type.
    numeric_features = X_train.select_dtypes(include=['number']).columns
    categorical_features = X_train.select_dtypes(include=['object']).columns

    # Pre-processor generation with multiple parameters that can be tweaked. I used the same values as LoGoFunc
    preprocessor = generate_preprocessor(numeric_features, categorical_features, 40, 0,
                                1, 0, 2, 0, 0, 1,
                                prefix='light0', do_feature_subset=True, max_features=1)

    
    if isinstance(y_train, list):
        y_train = pd.DataFrame({'label': y_train})

    X_train, y_train = preprocess(preprocessor, X_train, y_train)

    # This creates a directory for the preprocessor and trained models.
    joblib.dump(preprocessor, f'{model_path}/preprocessor.joblib')
    os.makedirs(model_path, exist_ok=True)

    def train_model(params, i, X_train, y_train):
        dtrain = Dataset(X_train.to_numpy(), y_train)
        lparams = {
            "learning_rate": params["learning_rate_" + str(i)],
            "min_data_in_leaf": params["min_data_in_leaf_" + str(i)],
            "num_leaves": params["num_leaves_" + str(i)],
            "min_child_weight": params["min_child_weight_" + str(i)],
            'max_depth': params["max_depth_" + str(i)],
            'colsample_bytree': params["colsample_bytree_" + str(i)],
            'subsample': params["subsample_" + str(i)],
            "lambda_l2": params["reg_lambda_" + str(i)],
            "lambda_l1": params["reg_alpha_" + str(i)],
            "bagging_freq": 1 if params["subsample_" + str(i)] != 1 else 0,
            "is_unbalance": True if params["is_unbalance_" + str(i)] else False,
            "num_iterations": params['num_iterations_' + str(i)],
            "num_threads": 10,
            "metric": 'multi_logloss',
            'objective': 'multiclassova',
            'num_class': 3,
            'verbosity': -1,
            'random_state': 41
        }
        model = lightgbm.train(lparams, dtrain)
        return model, lparams

    # Train the ensemble in parallel.
    models_and_params = Parallel(n_jobs=-1)(delayed(train_model)(params, i, X_train, y_train) for i in range(27))

    # Export models and params.
    for i, (model, lparams) in enumerate(models_and_params):
        joblib.dump(model, f'{model_path}/lgb_model_{i}.joblib')
        with open(f'{model_path}/lgb_params_{i}.json', 'w') as f:
            json.dump(lparams, f, indent=2)

    print(f"Training complete for {df_name}.")

print("Finished.")